In [1]:
import sqlite3
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import joblib

# === Step 1: Connect to DB & Read Data ===
conn = sqlite3.connect("energy_filled.db")
cursor = conn.cursor()

# Get table name
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_name = cursor.fetchall()[0][0]
print(f"Table found: {table_name}")

# Load data - include all relevant columns
query = f"""
    SELECT year, country, population, biofuel_consumption, renewables_consumption,
           coal_consumption, fossil_fuel_consumption, hydro_consumption,
           nuclear_consumption, oil_consumption, other_renewable_consumption,
           solar_consumption, wind_consumption
    FROM {table_name}
    WHERE renewables_consumption IS NOT NULL
"""
df = pd.read_sql_query(query, conn)
conn.close()

# Check initial data
if df.empty:
    print("Error: No data loaded from the database.")
    exit()

print(f"Initial data shape: {df.shape}")

# === Step 2: Preprocess & Create Lagged Feature ===
df.sort_values(by=["country", "year"], inplace=True)

# Calculate lagged consumption
df['lagged_renewables_consumption'] = df.groupby('country')['renewables_consumption'].shift(1)

# Handle NaN values resulting from the lag (first year per country)
initial_rows = df.shape[0]
df.dropna(subset=['lagged_renewables_consumption'], inplace=True)
print(f"Dropped {initial_rows - df.shape[0]} rows due to missing lagged consumption (first year per country).")

# === Step 3: Feature Engineering - Encode Country ===
# Get the list of all unique countries
all_countries = df['country'].unique()

# Initialize OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit encoder on the 'country' column
country_encoded = encoder.fit_transform(df[['country']])

# Create a DataFrame from the encoded country data
country_df = pd.DataFrame(country_encoded, columns=encoder.get_feature_names_out(['country']), index=df.index)

# Concatenate the encoded country features with the numerical features
df = pd.concat([df, country_df], axis=1)

# Drop the original 'country' column
df.drop('country', axis=1, inplace=True)

# === Step 4: Feature and Target Selection ===
# X includes year, lagged consumption, and one-hot encoded country columns
feature_columns = ['year', 'lagged_renewables_consumption'] + list(country_df.columns)
X = df[feature_columns]
y = df['renewables_consumption']

print(f"Features shape (X): {X.shape}")
print(f"Target shape (y): {y.shape}")
print(f"Features used ({len(feature_columns)}): {feature_columns}")

# === Step 5: Train/Test Split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Train shapes: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Test shapes: X_test={X_test.shape}, y_test={y_test.shape}")

# === Step 6: Train XGBoost Regressor (GPU or CPU) ===
try:
    model = XGBRegressor(
        tree_method="gpu_hist",
        predictor="gpu_predictor",
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        random_state=42
    )
    model.fit(X_train, y_train)
    print("\n✅ Model Trained (GPU)")
except Exception as e:
    print(f"\nGPU training failed ({e}). Falling back to CPU.")
    model = XGBRegressor(
        tree_method="hist",
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        random_state=42
    )
    model.fit(X_train, y_train)
    print("\n✅ Model Trained (CPU)")

# === Step 7: Evaluation ===
y_pred = model.predict(X_test)
print(f"📉 MSE: {mean_squared_error(y_test, y_pred):.2f}")
print(f"📈 R² Score: {r2_score(y_test, y_pred):.4f}")

# === Step 8: Save the Trained Model and Feature Columns ===
model_filename = 'renewable_energy_model.joblib'
joblib.dump(model, model_filename)
print(f"\n✅ Model saved to {model_filename}")

feature_columns_filename = 'model_feature_columns.joblib'
joblib.dump(feature_columns, feature_columns_filename)
print(f"✅ Feature column list saved to {feature_columns_filename}")

# Save the encoder
encoder_filename = 'country_encoder.joblib'
joblib.dump(encoder, encoder_filename)
print(f"✅ Encoder saved to {encoder_filename}")

# === Step 9: Helper function to create prediction input DataFrame with growth rate ===
def create_prediction_input_df(target_year, growth_rate, last_year, last_consumption, target_entity, all_feature_columns, encoder):
    """
    Creates a Pandas DataFrame row for prediction, applying a growth rate to calculate lagged consumption.

    Args:
        target_year (int): The year to predict renewable consumption for.
        growth_rate (float): Annual growth rate as a decimal (e.g., 0.05 for 5%).
        last_year (int): The last known year with data.
        last_consumption (float): The last known renewables consumption value.
        target_entity (str): The name of the country (e.g., 'United States').
        all_feature_columns (list): The exact list of feature column names the model was trained on.
        encoder (OneHotEncoder): The fitted encoder to transform the target entity.

    Returns:
        pandas.DataFrame: A DataFrame with one row ready for model.predict().
    """
    # Calculate lagged consumption for the year before the target year using the growth rate
    years_ahead = target_year - last_year - 1
    if years_ahead < 0:
        raise ValueError("Target year must be greater than the last known year.")
    lagged_consumption = last_consumption * (1 + growth_rate) ** years_ahead

    # Create a dictionary to hold the data for one row
    data = {}
    for col in all_feature_columns:
        data[col] = 0.0

    # Set the specific values
    data['year'] = target_year
    data['lagged_renewables_consumption'] = lagged_consumption

    # Encode the target entity
    entity_encoded = encoder.transform([[target_entity]])
    entity_df = pd.DataFrame(entity_encoded, columns=encoder.get_feature_names_out(['country']))
    for col in entity_df.columns:
        if col in data:
            data[col] = entity_df[col].iloc[0]
        else:
            print(f"Warning: Encoded column {col} not found in feature columns. Using 0.")

    # Create DataFrame ensuring column order matches training features
    prediction_df = pd.DataFrame([data], columns=all_feature_columns)

    return prediction_df

# === Step 10: Example Prediction using the helper function ===
# Load the necessary info
loaded_feature_columns = joblib.load('model_feature_columns.joblib')
loaded_model = joblib.load('renewable_energy_model.joblib')
loaded_encoder = joblib.load('country_encoder.joblib')

# Find the last known data point for a country
conn = sqlite3.connect("energy_filled.db")
last_known_data = pd.read_sql_query(query, conn).sort_values(by=["country", "year"])
last_known_data['lagged_renewables_consumption_temp'] = last_known_data.groupby('country')['renewables_consumption'].shift(1)
conn.close()

# Example Prediction: Predict for 'United States' with a growth rate
target_country = 'United States'
last_valid_point = last_known_data[(last_known_data['country'] == target_country) & (last_known_data['lagged_renewables_consumption_temp'].notna())].iloc[-1]
last_year_known = int(last_valid_point['year'])
last_consumption_known = last_valid_point['renewables_consumption']
growth_rate = 0.05  # 5% annual growth rate (example)
target_year = 2030

print(f"\n--- Prediction for {target_country} ---")
print(f"Last known year: {last_year_known}, Last consumption: {last_consumption_known:.2f}")
print(f"Target year: {target_year}, Growth rate: {growth_rate*100:.1f}%")

sample_input_df = create_prediction_input_df(
    target_year=target_year,
    growth_rate=growth_rate,
    last_year=last_year_known,
    last_consumption=last_consumption_known,
    target_entity=target_country,
    all_feature_columns=loaded_feature_columns,
    encoder=loaded_encoder
)

try:
    sample_output = loaded_model.predict(sample_input_df)[0]
    print(f"🔮 Predicted renewables consumption for {target_country} in {target_year}: {sample_output:.2f}")
except Exception as e:
    print(f"Error during prediction: {e}")

IndexError: list index out of range

In [ ]:
# Aggregate to create 'World' data
world_df = df.groupby('year').agg({
    'population': 'sum',
    'biofuel_consumption': 'sum',
    'renewables_consumption': 'sum',
    'coal_consumption': 'sum',
    'fossil_fuel_consumption': 'sum',
    'hydro_consumption': 'sum',
    'nuclear_consumption': 'sum',
    'oil_consumption': 'sum',
    'other_renewable_consumption': 'sum',
    'solar_consumption': 'sum',
    'wind_consumption': 'sum'
}).reset_index()
world_df['country'] = 'World'
df = pd.concat([df, world_df], ignore_index=True)

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import joblib

# === Step 1: Connect to DB & Read Data ===
conn = sqlite3.connect("energy_filled.db")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_name = cursor.fetchall()[0][0]
print(f"Table found: {table_name}")

query = f"""
    SELECT year, country, population, biofuel_consumption, renewables_consumption,
           coal_consumption, fossil_fuel_consumption, hydro_consumption,
           nuclear_consumption, oil_consumption, other_renewable_consumption,
           solar_consumption, wind_consumption
    FROM {table_name}
    WHERE renewables_consumption IS NOT NULL
      AND fossil_fuel_consumption IS NOT NULL
      AND coal_consumption IS NOT NULL
      AND oil_consumption IS NOT NULL
"""
df = pd.read_sql_query(query, conn)
conn.close()

if df.empty:
    print("Error: No data loaded from the database.")
    exit()

print(f"Initial data shape: {df.shape}")

# === Step 2: Preprocess & Create Features ===
df.sort_values(by=["country", "year"], inplace=True)

# Lagged features
df['lagged_renewables_consumption'] = df.groupby('country')['renewables_consumption'].shift(1)
df['non_renewable_consumption'] = df['fossil_fuel_consumption'] + df['coal_consumption'] + df['oil_consumption']
df['lagged_non_renewable_consumption'] = df.groupby('country')['non_renewable_consumption'].shift(1)

# Smooth growth rate with rolling average to reduce noise
def safe_pct_change(series):
    pct = series.pct_change()
    pct = pct.replace([np.inf, -np.inf], 0).fillna(0)
    return pct

df['renewable_growth_rate'] = safe_pct_change(df.groupby('country')['renewables_consumption'])
df['smoothed_growth_rate'] = df.groupby('country')['renewable_growth_rate'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)
df['lagged_smoothed_growth_rate'] = df.groupby('country')['smoothed_growth_rate'].shift(1).fillna(0)

# Add a trend feature (normalized year)
df['year_trend'] = (df['year'] - df['year'].min()) / (df['year'].max() - df['year'].min())

# Validate data
print("\nChecking for invalid values in key columns:")
for col in ['renewables_consumption', 'non_renewable_consumption', 'smoothed_growth_rate']:
    invalid = df[col].isna() | np.isinf(df[col]) | (df[col].abs() > 1e308)
    if invalid.any():
        print(f"Found {invalid.sum()} invalid values in {col}:")
        print(df[invalid][['year', 'country', col]].head())

# Drop invalid rows
initial_rows = df.shape[0]
df = df.dropna(subset=['lagged_renewables_consumption', 'lagged_non_renewable_consumption', 'lagged_smoothed_growth_rate'])
df = df[~df['renewables_consumption'].isna() & ~np.isinf(df['renewables_consumption'])]
df = df[~df['non_renewable_consumption'].isna() & ~np.isinf(df['non_renewable_consumption'])]
df = df[~df['smoothed_growth_rate'].isna() & ~np.isinf(df['smoothed_growth_rate'])]
print(f"Dropped {initial_rows - df.shape[0]} rows due to missing or invalid features/targets.")

if df.empty:
    print("Error: No valid data after cleaning.")
    exit()

# === Step 3: Feature Engineering - Encode Country ===
all_countries = df['country'].unique()
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
country_encoded = encoder.fit_transform(df[['country']])
country_df = pd.DataFrame(country_encoded, columns=encoder.get_feature_names_out(['country']), index=df.index)
df = pd.concat([df, country_df], axis=1)
df.drop('country', axis=1, inplace=True)

# === Step 4: Feature and Target Selection ===
base_features = ['year', 'year_trend', 'population'] + list(country_df.columns)

# Renewable consumption model
renewable_features = base_features + ['lagged_renewables_consumption']
X_renewable = df[renewable_features]
y_renewable = df['renewables_consumption']

# Non-renewable consumption model
non_renewable_features = base_features + ['lagged_non_renewable_consumption']
X_non_renewable = df[non_renewable_features]
y_non_renewable = df['non_renewable_consumption']

# Growth rate model
growth_features = base_features + ['lagged_renewables_consumption', 'lagged_smoothed_growth_rate']
X_growth = df[growth_features]
y_growth = df['smoothed_growth_rate']

# Validate targets
for name, y in [('Renewable', y_renewable), ('Non-Renewable', y_non_renewable), ('Growth', y_growth)]:
    invalid = y.isna() | np.isinf(y) | (y.abs() > 1e308)
    if invalid.any():
        print(f"Error: {name} target contains {invalid.sum()} invalid values.")
        exit()

print(f"Renewable model - Features: {renewable_features}")
print(f"Non-renewable model - Features: {non_renewable_features}")
print(f"Growth rate model - Features: {growth_features}")

# === Step 5: Train/Test Split ===
X_renew_train, X_renew_test, y_renew_train, y_renew_test = train_test_split(X_renewable, y_renewable, test_size=0.2, random_state=42)
X_non_renew_train, X_non_renew_test, y_non_renew_train, y_non_renew_test = train_test_split(X_non_renewable, y_non_renewable, test_size=0.2, random_state=42)
X_growth_train, X_growth_test, y_growth_train, y_growth_test = train_test_split(X_growth, y_growth, test_size=0.2, random_state=42)

# === Step 6: Train Models ===
def train_xgboost(X_train, y_train, model_name):
    try:
        model = XGBRegressor(
            tree_method="gpu_hist",
            predictor="gpu_predictor",
            n_estimators=100,
            max_depth=5,
            learning_rate=0.1,
            random_state=42
        )
        model.fit(X_train, y_train)
        print(f"\n✅ {model_name} Trained (GPU)")
    except Exception as e:
        print(f"\nGPU training failed for {model_name} ({e}). Falling back to CPU.")
        model = XGBRegressor(
            tree_method="hist",
            n_estimators=100,
            max_depth=5,
            learning_rate=0.1,
            random_state=42
        )
        model.fit(X_train, y_train)
        print(f"\n✅ {model_name} Trained (CPU)")
    return model

renewable_model = train_xgboost(X_renew_train, y_renew_train, "Renewable Model")
non_renewable_model = train_xgboost(X_non_renew_train, y_non_renew_train, "Non-Renewable Model")
growth_model = train_xgboost(X_growth_train, y_growth_train, "Growth Rate Model")

# === Step 7: Evaluate Models ===
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{model_name} - MSE: {mse:.2f}, R²: {r2:.4f}")

evaluate_model(renewable_model, X_renew_test, y_renew_test, "Renewable Model")
evaluate_model(non_renewable_model, X_non_renew_test, y_non_renew_test, "Non-Renewable Model")
evaluate_model(growth_model, X_growth_test, y_growth_test, "Growth Rate Model")

# === Step 8: Save Models and Metadata ===
joblib.dump(renewable_model, 'renewable_energy_model.joblib')
joblib.dump(non_renewable_model, 'non_renewable_energy_model.joblib')
joblib.dump(growth_model, 'growth_rate_model.joblib')
joblib.dump(encoder, 'country_encoder.joblib')
joblib.dump(renewable_features, 'renewable_feature_columns.joblib')
joblib.dump(non_renewable_features, 'non_renewable_feature_columns.joblib')
joblib.dump(growth_features, 'growth_feature_columns.joblib')
print("\n✅ All models and metadata saved.")

# === Step 9: Helper Functions for Prediction ===
def create_prediction_input_df(target_year, last_year, last_value, target_entity, feature_columns, encoder, model_type, last_growth_rate=None, last_population=None):
    data = {col: 0.0 for col in feature_columns}
    data['year'] = target_year
    data['year_trend'] = (target_year - 1980) / (2050 - 1980)  # Normalize based on historical range

    if model_type == 'renewable':
        data['lagged_renewables_consumption'] = last_value
    elif model_type == 'non_renewable':
        data['lagged_non_renewable_consumption'] = last_value
    elif model_type == 'growth':
        data['lagged_renewables_consumption'] = last_value
        data['lagged_smoothed_growth_rate'] = last_growth_rate if last_growth_rate is not None else 0.0

    # Extrapolate population
    if last_population is not None:
        years_diff = target_year - last_year
        population_growth_rate = 0.01  # Assume 1% annual growth
        data['population'] = last_population * (1 + population_growth_rate) ** years_diff

    entity_df = pd.DataFrame([[target_entity]], columns=['country'])
    entity_encoded = encoder.transform(entity_df)
    encoded_df = pd.DataFrame(entity_encoded, columns=encoder.get_feature_names_out(['country']))
    for col in encoded_df.columns:
        if col in data:
            data[col] = encoded_df[col].iloc[0]

    return pd.DataFrame([data], columns=feature_columns)

Table found: filled_energy_data
Initial data shape: (13755, 13)

Checking for invalid values in key columns:
Dropped 125 rows due to missing or invalid features/targets.
Renewable model - Features: ['year', 'population', 'country_ASEAN (Ember)', 'country_Afghanistan', 'country_Africa', 'country_Africa (EI)', 'country_Africa (Ember)', 'country_Africa (Shift)', 'country_Albania', 'country_Algeria', 'country_American Samoa', 'country_Angola', 'country_Antarctica', 'country_Antigua and Barbuda', 'country_Argentina', 'country_Armenia', 'country_Aruba', 'country_Asia', 'country_Asia & Oceania (EIA)', 'country_Asia (Ember)', 'country_Asia Pacific (EI)', 'country_Asia and Oceania (Shift)', 'country_Australia', 'country_Australia and New Zealand (EIA)', 'country_Austria', 'country_Azerbaijan', 'country_Bahamas', 'country_Bahrain', 'country_Bangladesh', 'country_Barbados', 'country_Belarus', 'country_Belgium', 'country_Belize', 'country_Benin', 'country_Bermuda', 'country_Bhutan', 'country_Boliv

c:\renew\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:52:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
c:\renew\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:52:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



✅ Renewable Model Trained (GPU)


c:\renew\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:52:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
c:\renew\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:52:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



✅ Non-Renewable Model Trained (GPU)


c:\renew\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:52:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
c:\renew\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:52:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



✅ Growth Rate Model Trained (GPU)
Renewable Model - MSE: 4231.34, R²: 0.9979
Non-Renewable Model - MSE: 701864.07, R²: 0.9987
Growth Rate Model - MSE: 3.29, R²: -3.1860

✅ All models and metadata saved.


c:\renew\venv\Lib\site-packages\xgboost\core.py:2676: UserWarning: [09:52:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:
c:\renew\venv\Lib\site-packages\xgboost\core.py:729: UserWarning: [09:52:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)
c:\renew\venv\Lib\site-packages\xgboost\cor